In [1]:
import sys
import pickle
import codecs
sys.path.append('mlp_trainer')

# import GCP Demo 1 package
import trainer.model as model
import trainer.data.bigquery_generator as generator
import tensorflow as tf
from google.cloud import bigquery_storage_v1beta1
tf.compat.v1.enable_eager_execution()

Using TensorFlow backend.


In [2]:
dataset = generator.get_data("finaltaxi_encoded_sampled", "test", 64, 1, 50, 4)

In [3]:
count = 0
for value in dataset.take(1):
    print(value)
    count += 1 

.3125688 , -0.1454222 ,  0.54075485,  1.1713213 , -0.8727393 ,
       -0.18040498,  0.54075485,  1.1713213 ,  0.05927713, -0.2050552 ,
        0.54075485,  1.1713213 ,  0.05927713, -0.24513097,  0.54075485,
        0.87769055,  0.05927713, -0.30128413,  0.54075485], dtype=float32)>, <tf.Tensor: id=292, shape=(64,), dtype=float32, numpy=
array([ 0.11205149,  0.0779883 , -0.01440587,  0.02671614,  0.04481309,
       -0.06532092, -0.03780149,  0.02671614, -0.09492952, -0.06532092,
       -0.05868518,  0.02671614, -0.09492952, -0.06532092, -0.08880955,
        0.02671614,  0.19326666, -0.06532092,  0.02756678,  0.02671614,
        0.19326666, -0.06532092, -0.03797746,  0.02671614,  0.19326666,
        0.10502314, -0.03797746,  0.02671614,  0.19326666,  0.10502314,
       -0.02275077,  0.02671614,  0.19326666,  0.10502314, -0.02275077,
        0.02671614,  0.19326666, -0.08853708, -0.06677336,  0.02671614,
        0.19326666, -0.08853708,  0.00523258,  0.02671614,  0.15422358,
       -0.022

In [ ]:
def get_reader_for_stream(session_pickled: bytes, stream_name_bytes: bytes):
    session = pickle.loads(codecs.decode(session_pickled, "base64"))
    stream_name = stream_name_bytes.decode("utf-8")
    client = bigquery_storage_v1beta1.BigQueryStorageClient()
    for stream in session.streams:
        if stream.name == stream_name:
            reader = data.get_reader(client, stream)
            rows = reader.rows(session)
            for row in rows:
                # df = pd.DataFrame([row])
                yield (
                    # df.drop(['cash', ], axis=1).values,
                    # df['cash'].values 
                    [
                        row.get("year"),
                        row.get("start_time_norm_midnight"),
                        row.get("start_time_norm_noon"),
                        row.get("pickup_lat_std"),
                        row.get("pickup_long_std"),
                        row.get("pickup_lat_centered"),
                        row.get("pickup_long_centered"),
                        row.get("day_of_week_MONDAY"),
                        row.get("day_of_week_TUESDAY"),
                        row.get("day_of_week_WEDNESDAY"),
                        row.get("day_of_week_THURSDAY"),
                        row.get("day_of_week_FRIDAY"),
                        row.get("day_of_week_SATURDAY"),
                        row.get("day_of_week_SUNDAY"),
                        row.get("month_JANUARY"),
                        row.get("month_FEBRUARY"),
                        row.get("month_MARCH"),
                        row.get("month_APRIL"),
                        row.get("month_MAY"),
                        row.get("month_JUNE"),
                        row.get("month_JULY"),
                        row.get("month_AUGUST"),
                        row.get("month_SEPTEMBER"),
                        row.get("month_OCTOBER"),
                        row.get("month_NOVEMBER"),
                        row.get("month_DECEMBER"),
                    ],
                    row.get("cash")
                )

def bq_stream_generator(table_id: bytes, partition: bytes):
    session, _ = data.get_data_partition_sharded(table_id.decode("utf-8"), partition.decode("utf-8"), shards=100)
    encoded_session = codecs.encode(pickle.dumps(session), "base64")
    for stream in session.streams:
        yield(encoded_session, stream.name)

dataset = tf.data.Dataset.from_generator(
    bq_stream_generator,
    (tf.string, tf.string),
    # output_shapes=(tf.TensorShape([]), tf.TensorShape([])),
    args = (            "finaltaxi_encoded_sampled",
        'train')
).interleave(lambda session, stream: 
    tf.data.Dataset.from_generator(
        get_reader_for_stream,
        (tf.float64, tf.uint16),
        args=(session, stream)
    ).prefetch(buffer_size=tf.data.experimental.AUTOTUNE),
    num_parallel_calls=tf.data.experimental.AUTOTUNE
).prefetch(buffer_size=tf.data.experimental.AUTOTUNE).batch(2).repeat(2)

count = 0
for value in dataset.take(1):
    # if count % 100000 == 0:
    print(value)
    count += 1 